In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

a = chat.predict("How many planets are there?")  # 모델 객체.predict("질문 내용")을 통해 모델에 질문을 함.

a 
# 'As of now, there are officially eight planets in our solar system. These planets are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, it is important to note that there is ongoing debate and discussion regarding the status of Pluto as a planet. Pluto was reclassified as a "dwarf planet" by the International Astronomical Union (IAU) in 2006.'

'As of now, there are officially eight planets in our solar system. These planets are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, it is important to note that there is ongoing debate and discussion regarding the status of Pluto as a planet. Pluto was reclassified as a "dwarf planet" by the International Astronomical Union (IAU) in 2006.'

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1) # temperature속성을 통해 모델의 창의력을 설정해줌.

messages = [ # 메세지 리스트
    SystemMessage(content="You are geography expert. And you only reply in Korean."), # SystemMessage(content="모델 설정 내용")
    AIMessage(content="Hello, My Name is Paul!"), # AIMessage(content="시작 인사말 내용")
    HumanMessage(content="What is the distance between Mexico and Thailand?"), # HumanMessage(content="실제 질문 내용")
]

a = chat.predict_messages(messages) # 모델 객체.predict_messages(메세지 리스트)를 통해 모델에 질문을 함. 

a
# AIMessage(content='안녕하세요, 제 이름은 폴이에요! 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다.')

AIMessage(content='안녕하세요! 저는 지리 전문가입니다. 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다. 제 이름은 폴이에요! 어떻게 도와드릴까요?')

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(temperature=0.1) 

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
) # PromptTemplate.from_template("질문 내용 {변수명}")을 통해 템플릿 객체를 생성함.

prompt = template.format(country_a="Mexico", country_b="Thailand") # 템플릿 객체.format(변수명="변수값")를 통해 프롬프트를 생성함.


a = chat.predict(prompt) # 모델 객체.predict(프롬프트)를 통해 모델에 질문을 함.

a
# 'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1) 

template = ChatPromptTemplate.from_messages( # ChatPromptTemplate.from_messages([ 메세지 튜플{변수명} ])을 통해 템플릿 객체를 생성함.
    [ 
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Hello, my name is {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}"),
    ]
)

ChatPrompt = template.format_messages( # 템플릿 객체.format_messages(변수명="변수값")을 통해 챗 프롬프트를 생성함.
    language="Korean",
    name="Paul",
    country_a="Mexico",
    country_b="Thailand"
)

a = chat.predict_messages(ChatPrompt) # 모델 객체.predict_messages(챗 프롬프트)를 통해 모델에 질문을 함.

a
# AIMessage(content='안녕하세요, 제 이름은 폴이에요! 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다.')

AIMessage(content='멕시코와 태국 사이의 거리는 약 17,000 킬로미터입니다.')

In [ ]:
############################################

In [3]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items)) 


p = CommaOutputParser()
p.parse("Hello, how, are, you")

# ['Hello', 'how', 'are', 'you']

['Hello', 'how', 'are', 'you']

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1) 


template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do not reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(max_items=10, question="What are the colors?")

result = chat.predict_messages(prompt) 
result  # AIMessage(content='red, orange, yellow, green, blue, indigo, violet, black, white, gray')

p = CommaOutputParser()

parsedResult = p.parse(result.content)

parsedResult

# ['red',
#  'orange',
#  'yellow',
#  'green',
#  'blue',
#  'indigo',
#  'violet',
#  'black',
#  'white',
#  'gray']

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [11]:
chain = template | chat | CommaOutputParser()

chain.invoke ({
    "max_items": 10,
    "question": "What are the colors?"
})

# ['red',
#  'orange',
#  'yellow',
#  'green',
#  'blue',
#  'indigo',
#  'violet',
#  'black',
#  'white',
#  'gray']

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 


chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])


chef_chain = chef_template | chat


In [ ]:
veg_template = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_template | chat

final_chain = {"recipe":chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "Korean"
})


"""
Great choice! Korean cuisine is known for its bold flavors and unique combinations. Here's a recipe for a classic Korean dish called Bibimbap:

Ingredients:
- 2 cups cooked rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup spinach
- 1 cup bean sprouts
- 4 shiitake mushrooms, sliced
- 1/2 pound beef (can be substituted with tofu or chicken), thinly sliced
- 4 eggs
- 2 tablespoons vegetable oil
- 2 tablespoons soy sauce
- 1 tablespoon sesame oil
- 1 tablespoon gochujang (Korean chili paste)
- Salt, to taste
- Sesame seeds, for garnish
- Kimchi, for serving (optional)

Instructions:
1. Cook the rice according to the package instructions and set aside.
2. Blanch the spinach and bean sprouts in boiling water for 1-2 minutes. Drain and rinse with cold water. Squeeze out any excess water from the spinach and season with a pinch of salt and 1 teaspoon of sesame oil. Set aside.
3. Heat 1 tablespoon of vegetable oil in a pan over medium heat. Stir-fry the carrots and zucchini for 2-3 minutes until slightly softened. Season with a pinch of salt and set aside.
4. In the same pan, add another tablespoon of vegetable oil and cook the mushrooms until they are tender. Season with soy sauce and set aside.
5. In a separate pan, cook the beef (or tofu/chicken) over medium-high heat until browned and cooked through. Season with soy sauce and set aside.
6. In the same pan, fry the eggs sunny-side up or to your desired doneness.
7. To assemble the bibimbap, divide the rice into four bowls. Arrange the cooked vegetables and beef (or tofu/chicken) on top of the rice.
8. In a small bowl, mix together gochujang, 1 tablespoon of soy sauce, and 1 tablespoon of sesame oil. Drizzle this sauce over the rice and vegetables.
9. Top each bowl with a fried egg and sprinkle with sesame seeds.
10. Serve the bibimbap with kimchi on the side, if desired.

Enjoy your homemade Korean Bibimbap!Great choice! Bibimbap is a delicious Korean dish that can easily be made vegetarian. Here's how you can make a vegetarian version of Bibimbap:

Ingredients:
- 2 cups cooked rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup spinach
- 1 cup bean sprouts
- 4 shiitake mushrooms, sliced
- 1/2 pound tofu, thinly sliced
- 4 eggs (optional, omit for a vegan version)
- 2 tablespoons vegetable oil
- 2 tablespoons soy sauce
- 1 tablespoon sesame oil
- 1 tablespoon gochujang (Korean chili paste)
- Salt, to taste
- Sesame seeds, for garnish
- Kimchi, for serving (optional)

Instructions:
1. Cook the rice according to the package instructions and set aside.
2. Blanch the spinach and bean sprouts in boiling water for 1-2 minutes. Drain and rinse with cold water. Squeeze out any excess water from the spinach and season with a pinch of salt and 1 teaspoon of sesame oil. Set aside.
3. Heat 1 tablespoon of vegetable oil in a pan over medium heat. Stir-fry the carrots and zucchini for 2-3 minutes until slightly softened. Season with a pinch of salt and set aside.
4. In the same pan, add another tablespoon of vegetable oil and cook the mushrooms until they are tender. Season with soy sauce and set aside.
5. In a separate pan, cook the tofu over medium-high heat until browned and cooked through. Season with soy sauce and set aside.
6. In a separate pan, fry the eggs sunny-side up or to your desired doneness (omit for a vegan version).
7. To assemble the bibimbap, divide the rice into four bowls. Arrange the cooked vegetables and tofu on top of the rice.
8. In a small bowl, mix together gochujang, 1 tablespoon of soy sauce, and 1 tablespoon of sesame oil. Drizzle this sauce over the rice and vegetables.
9. Top each bowl with a fried egg (if using) and sprinkle with sesame seeds.
10. Serve the bibimbap with kimchi on the side, if desired.

Enjoy your homemade vegetarian Korean Bibimbap!

"""

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 

t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
)

prompt = t.format(country="France")

chat.predict(prompt)

# The capital of France is Paris.


In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 



examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_template = """
    Q: {question}
    A: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)


prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)


chain = prompt | chat

chain.invoke({
    "country": "Korea"
})

"""
Teacher: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won
"""

Teacher: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='Teacher: \nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 



examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {   "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]




example_prompt = ChatPromptTemplate.from_messages([
    ("Q", "what do you know about {country}?"),
    ("A", "{answer}")
])


example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)


final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you know about {country}")
])


chain = final_prompt | chat

chain.invoke({
    "country": "USA"
})


"""
  I know this:
        Capital: Washington, D.C.
        Language: English
        Food: Hamburgers and Hotdogs
        Currency: United States Dollar (USD)
"""


        I know this:
        Capital: Washington, D.C.
        Language: English
        Food: Hamburgers and Hotdogs
        Currency: United States Dollar (USD)
        

AIMessageChunk(content='\n        I know this:\n        Capital: Washington, D.C.\n        Language: English\n        Food: Hamburgers and Hotdogs\n        Currency: United States Dollar (USD)\n        ')

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]





example_template = """
    Q: {question}
    A: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)


example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt=example_prompt,
    max_length=100,
)


prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix="Q: What do you know about {country}?",
    input_variables=["country"]
)


prompt.format(country="Brazil")

# '\n    Q: What do you know about France?\n    A: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\nQ: What do you know about Brazil?'


'\n    Q: What do you know about France?\n    A: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\nQ: What do you know about Brazil?'

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]



class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]





example_template = """
    Q: {question}
    A: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)


example_selector = RandomExampleSelector(
    examples = examples,
)


prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix="Q: What do you know about {country}?",
    input_variables=["country"]
)


prompt.format(country="Brazil")
# '\n    Q: What do you know about France?\n    A: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\nQ: What do you know about Brazil?'


'\n    Q: What do you know about Italy?\n    A: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\n\nQ: What do you know about Brazil?'